In [1]:
from glob import glob
import os
import pandas as pd
import torch
from bunch import Bunch
from ruamel.yaml import safe_load
from torch.utils.data import DataLoader


from tester import Tester
import networks as models
from datasets.fives import FIVES
from utils.helpers import get_instance
from utils import losses

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
data_path = '/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/datasets/FIVES'
path_name = f"{data_path}/Quality Assessment.xlsx"
df = pd.read_excel(path_name, sheet_name='Test')
df['Score'] = df['IC'] + df['Blur'] + df['LC']

# read all image paths to a dataframe
images = sorted(glob(os.path.join(data_path, 'test/Original/*')))
labels = sorted(glob(os.path.join(data_path, 'test/Ground truth/*')))

path_df = pd.DataFrame({'images':images, 'labels':labels})
path_df['Number'] = path_df['images'].apply(lambda x: int(x.split('/')[-1].split('_')[0]))

new_df = pd.merge(df, path_df, on='Number')

In [4]:
def load_images(df, quality, value):
    # Given the dataframe,
    # retun images and columns for a specific group based
    # IC == 0

    assert value in [0,1,2,3], "Specify the correct value"
    assert quality in ["IC", "Blur", "LC", "Score"]

    group = df[df[quality] == value]
    grouped = group[['images', 'labels']]
    group_images = grouped['images'].values.tolist()
    group_labels = grouped['labels'].values.tolist()

    return group_images, group_labels

    

In [5]:
def main_result(config_file, weight_path, quality, value):

    with open(config_file, encoding="utf-8") as file:
        CFG = Bunch(safe_load(file))

    
    valid_x, valid_y  = load_images(new_df, quality=quality, value=value)


    dataset = FIVES(CFG=CFG, images_path=valid_x, mask_paths=valid_y, mode='test')

    checkpoint = torch.load(weight_path)

    CFG_ck = checkpoint['config']

    test_loader = DataLoader(dataset, 1, shuffle=False,  
                            num_workers=CFG.num_workers, pin_memory=True)
    model = get_instance(models, 'model', CFG)
    loss = get_instance(losses, 'loss', CFG_ck)

    test = Tester(model, loss, CFG, checkpoint, test_loader, device)
    test.test()


## FR-UNet

In [6]:
config_file = '../configs/fr_unet_dicebce.yaml'
check_path = "/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/FR_UNet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth"


In [7]:
main_result(config_file, weight_path=check_path, quality="Score", value=0)

main_result(config_file, weight_path=check_path, quality="Score", value=1)

main_result(config_file, weight_path=check_path, quality="Score", value=2)

main_result(config_file, weight_path=check_path, quality="Score", value=3)

TEST (14) | Loss: 0.2319 | AUC 0.9388 F1 0.6377 Acc 0.9863  Sen 0.5370 Spe 0.9979 Pre 0.8563 IOU 0.4999 MCC 0.6566 |B 0.31 D 0.13 |: 100%|██████████| 15/15 [00:06<00:00,  2.31it/s]
2023-08-14 12:09:42.205 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:09:42.206 | INFO     | tester:test:79 - test time:  0.3069
2023-08-14 12:09:42.207 | INFO     | tester:test:80 -      loss:  0.2319
2023-08-14 12:09:42.207 | INFO     | tester:test:84 - AUC  : 0.9388
2023-08-14 12:09:42.208 | INFO     | tester:test:84 - F1   : 0.6377
2023-08-14 12:09:42.209 | INFO     | tester:test:84 - Acc  : 0.9863
2023-08-14 12:09:42.209 | INFO     | tester:test:84 - Sen  : 0.537
2023-08-14 12:09:42.209 | INFO     | tester:test:84 - Spe  : 0.9979
2023-08-14 12:09:42.211 | INFO     | tester:test:84 - pre  : 0.8563
2023-08-14 12:09:42.211 | INFO     | tester:test:84 - IOU  : 0.4999
2023-08-14 12:09:42.212 | INFO     | tester:test:84 - MCC  : 0.6566
TEST (10) | Loss: 0.1339 | AUC 0.9855 F1 0.823

In [8]:
main_result(config_file, weight_path=check_path, quality="IC", value=0)

main_result(config_file, weight_path=check_path, quality="IC", value=1)


TEST (51) | Loss: 0.1395 | AUC 0.9749 F1 0.8062 Acc 0.9857  Sen 0.7388 Spe 0.9968 Pre 0.9164 IOU 0.6998 MCC 0.8086 |B 0.09 D 0.02 |: 100%|██████████| 52/52 [00:11<00:00,  4.45it/s]
2023-08-14 12:10:38.648 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:10:38.649 | INFO     | tester:test:79 - test time:  0.0921
2023-08-14 12:10:38.650 | INFO     | tester:test:80 -      loss:  0.1395
2023-08-14 12:10:38.650 | INFO     | tester:test:84 - AUC  : 0.9749
2023-08-14 12:10:38.651 | INFO     | tester:test:84 - F1   : 0.8062
2023-08-14 12:10:38.651 | INFO     | tester:test:84 - Acc  : 0.9857
2023-08-14 12:10:38.652 | INFO     | tester:test:84 - Sen  : 0.7388
2023-08-14 12:10:38.652 | INFO     | tester:test:84 - Spe  : 0.9968
2023-08-14 12:10:38.652 | INFO     | tester:test:84 - pre  : 0.9164
2023-08-14 12:10:38.653 | INFO     | tester:test:84 - IOU  : 0.6998
2023-08-14 12:10:38.653 | INFO     | tester:test:84 - MCC  : 0.8086
TEST (147) | Loss: 0.0759 | AUC 0.9932 F1 0.9

In [9]:
main_result(config_file, weight_path=check_path, quality="Blur", value=0)

main_result(config_file, weight_path=check_path, quality="Blur", value=1)


TEST (40) | Loss: 0.1601 | AUC 0.9704 F1 0.7693 Acc 0.9867  Sen 0.6957 Spe 0.9971 Pre 0.8958 IOU 0.6500 MCC 0.7745 |B 0.09 D 0.03 |: 100%|██████████| 41/41 [00:09<00:00,  4.36it/s]
2023-08-14 12:11:20.260 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:11:20.260 | INFO     | tester:test:79 - test time:  0.0939
2023-08-14 12:11:20.261 | INFO     | tester:test:80 -      loss:  0.1601
2023-08-14 12:11:20.262 | INFO     | tester:test:84 - AUC  : 0.9704
2023-08-14 12:11:20.262 | INFO     | tester:test:84 - F1   : 0.7693
2023-08-14 12:11:20.263 | INFO     | tester:test:84 - Acc  : 0.9867
2023-08-14 12:11:20.264 | INFO     | tester:test:84 - Sen  : 0.6957
2023-08-14 12:11:20.264 | INFO     | tester:test:84 - Spe  : 0.9971
2023-08-14 12:11:20.264 | INFO     | tester:test:84 - pre  : 0.8958
2023-08-14 12:11:20.265 | INFO     | tester:test:84 - IOU  : 0.65
2023-08-14 12:11:20.265 | INFO     | tester:test:84 - MCC  : 0.7745
TEST (158) | Loss: 0.0750 | AUC 0.9931 F1 0.915

In [10]:
main_result(config_file, weight_path=check_path, quality="LC", value=0)

main_result(config_file, weight_path=check_path, quality="LC", value=1)


TEST (14) | Loss: 0.2319 | AUC 0.9388 F1 0.6377 Acc 0.9863  Sen 0.5370 Spe 0.9979 Pre 0.8563 IOU 0.4999 MCC 0.6566 |B 0.16 D 0.09 |: 100%|██████████| 15/15 [00:04<00:00,  3.43it/s]
2023-08-14 12:11:59.037 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:11:59.038 | INFO     | tester:test:79 - test time:  0.1555
2023-08-14 12:11:59.038 | INFO     | tester:test:80 -      loss:  0.2319
2023-08-14 12:11:59.039 | INFO     | tester:test:84 - AUC  : 0.9388
2023-08-14 12:11:59.039 | INFO     | tester:test:84 - F1   : 0.6377
2023-08-14 12:11:59.040 | INFO     | tester:test:84 - Acc  : 0.9863
2023-08-14 12:11:59.040 | INFO     | tester:test:84 - Sen  : 0.537
2023-08-14 12:11:59.041 | INFO     | tester:test:84 - Spe  : 0.9979
2023-08-14 12:11:59.041 | INFO     | tester:test:84 - pre  : 0.8563
2023-08-14 12:11:59.042 | INFO     | tester:test:84 - IOU  : 0.4999
2023-08-14 12:11:59.042 | INFO     | tester:test:84 - MCC  : 0.6566
TEST (184) | Loss: 0.0811 | AUC 0.9925 F1 0.90

## SAUNet

In [11]:
config_file = '../configs/sa_unet_dicebce.yaml'
check_path = "/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/SA_UNet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth"


In [12]:
main_result(config_file, weight_path=check_path, quality="Score", value=0)

main_result(config_file, weight_path=check_path, quality="Score", value=1)

main_result(config_file, weight_path=check_path, quality="Score", value=2)

main_result(config_file, weight_path=check_path, quality="Score", value=3)

TEST (14) | Loss: 0.3116 | AUC 0.9186 F1 0.6551 Acc 0.9848  Sen 0.6283 Spe 0.9935 Pre 0.7115 IOU 0.5130 MCC 0.6546 |B 0.08 D 0.07 |: 100%|██████████| 15/15 [00:03<00:00,  4.46it/s]
2023-08-14 12:12:41.441 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:12:41.442 | INFO     | tester:test:79 - test time:  0.082
2023-08-14 12:12:41.443 | INFO     | tester:test:80 -      loss:  0.3116
2023-08-14 12:12:41.443 | INFO     | tester:test:84 - AUC  : 0.9186
2023-08-14 12:12:41.444 | INFO     | tester:test:84 - F1   : 0.6551
2023-08-14 12:12:41.444 | INFO     | tester:test:84 - Acc  : 0.9848
2023-08-14 12:12:41.445 | INFO     | tester:test:84 - Sen  : 0.6283
2023-08-14 12:12:41.445 | INFO     | tester:test:84 - Spe  : 0.9935
2023-08-14 12:12:41.446 | INFO     | tester:test:84 - pre  : 0.7115
2023-08-14 12:12:41.446 | INFO     | tester:test:84 - IOU  : 0.513
2023-08-14 12:12:41.446 | INFO     | tester:test:84 - MCC  : 0.6546
TEST (10) | Loss: 0.2272 | AUC 0.9712 F1 0.8021

In [13]:
main_result(config_file, weight_path=check_path, quality="IC", value=0)

main_result(config_file, weight_path=check_path, quality="IC", value=1)


TEST (51) | Loss: 0.2217 | AUC 0.9632 F1 0.7916 Acc 0.9813  Sen 0.8090 Spe 0.9881 Pre 0.7866 IOU 0.6725 MCC 0.7849 |B 0.03 D 0.02 |: 100%|██████████| 52/52 [00:08<00:00,  5.98it/s]
2023-08-14 12:13:21.234 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:13:21.235 | INFO     | tester:test:79 - test time:  0.0299
2023-08-14 12:13:21.235 | INFO     | tester:test:80 -      loss:  0.2217
2023-08-14 12:13:21.236 | INFO     | tester:test:84 - AUC  : 0.9632
2023-08-14 12:13:21.236 | INFO     | tester:test:84 - F1   : 0.7916
2023-08-14 12:13:21.236 | INFO     | tester:test:84 - Acc  : 0.9813
2023-08-14 12:13:21.237 | INFO     | tester:test:84 - Sen  : 0.809
2023-08-14 12:13:21.238 | INFO     | tester:test:84 - Spe  : 0.9881
2023-08-14 12:13:21.238 | INFO     | tester:test:84 - pre  : 0.7866
2023-08-14 12:13:21.239 | INFO     | tester:test:84 - IOU  : 0.6725
2023-08-14 12:13:21.239 | INFO     | tester:test:84 - MCC  : 0.7849
TEST (147) | Loss: 0.1634 | AUC 0.9882 F1 0.87

In [14]:
main_result(config_file, weight_path=check_path, quality="Blur", value=0)

main_result(config_file, weight_path=check_path, quality="Blur", value=1)


TEST (40) | Loss: 0.2544 | AUC 0.9551 F1 0.7528 Acc 0.9829  Sen 0.7647 Spe 0.9904 Pre 0.7607 IOU 0.6221 MCC 0.7489 |B 0.04 D 0.03 |: 100%|██████████| 41/41 [00:07<00:00,  5.71it/s]
2023-08-14 12:13:51.211 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:13:51.212 | INFO     | tester:test:79 - test time:  0.0394
2023-08-14 12:13:51.213 | INFO     | tester:test:80 -      loss:  0.2544
2023-08-14 12:13:51.213 | INFO     | tester:test:84 - AUC  : 0.9551
2023-08-14 12:13:51.214 | INFO     | tester:test:84 - F1   : 0.7528
2023-08-14 12:13:51.214 | INFO     | tester:test:84 - Acc  : 0.9829
2023-08-14 12:13:51.215 | INFO     | tester:test:84 - Sen  : 0.7647
2023-08-14 12:13:51.215 | INFO     | tester:test:84 - Spe  : 0.9904
2023-08-14 12:13:51.216 | INFO     | tester:test:84 - pre  : 0.7607
2023-08-14 12:13:51.216 | INFO     | tester:test:84 - IOU  : 0.6221
2023-08-14 12:13:51.217 | INFO     | tester:test:84 - MCC  : 0.7489
TEST (158) | Loss: 0.1590 | AUC 0.9886 F1 0.8

In [15]:
main_result(config_file, weight_path=check_path, quality="LC", value=0)

main_result(config_file, weight_path=check_path, quality="LC", value=1)


TEST (14) | Loss: 0.3116 | AUC 0.9186 F1 0.6551 Acc 0.9848  Sen 0.6283 Spe 0.9935 Pre 0.7115 IOU 0.5130 MCC 0.6546 |B 0.09 D 0.08 |: 100%|██████████| 15/15 [00:03<00:00,  4.41it/s]
2023-08-14 12:14:19.264 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:14:19.265 | INFO     | tester:test:79 - test time:  0.0869
2023-08-14 12:14:19.265 | INFO     | tester:test:80 -      loss:  0.3116
2023-08-14 12:14:19.266 | INFO     | tester:test:84 - AUC  : 0.9186
2023-08-14 12:14:19.266 | INFO     | tester:test:84 - F1   : 0.6551
2023-08-14 12:14:19.266 | INFO     | tester:test:84 - Acc  : 0.9848
2023-08-14 12:14:19.267 | INFO     | tester:test:84 - Sen  : 0.6283
2023-08-14 12:14:19.268 | INFO     | tester:test:84 - Spe  : 0.9935
2023-08-14 12:14:19.268 | INFO     | tester:test:84 - pre  : 0.7115
2023-08-14 12:14:19.269 | INFO     | tester:test:84 - IOU  : 0.513
2023-08-14 12:14:19.270 | INFO     | tester:test:84 - MCC  : 0.6546
TEST (184) | Loss: 0.1678 | AUC 0.9868 F1 0.86

## MANet

In [16]:
config_file = '../configs/manet_dicebce.yaml'
check_path = "/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/MANet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth"


In [17]:
main_result(config_file, weight_path=check_path, quality="Score", value=0)

main_result(config_file, weight_path=check_path, quality="Score", value=1)

main_result(config_file, weight_path=check_path, quality="Score", value=2)

main_result(config_file, weight_path=check_path, quality="Score", value=3)

TEST (14) | Loss: 0.1853 | AUC 0.9631 F1 0.6995 Acc 0.9875  Sen 0.6492 Spe 0.9958 Pre 0.8023 IOU 0.5645 MCC 0.7054 |B 0.10 D 0.08 |: 100%|██████████| 15/15 [00:03<00:00,  4.22it/s]
2023-08-14 12:14:57.718 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:14:57.718 | INFO     | tester:test:79 - test time:  0.0958
2023-08-14 12:14:57.719 | INFO     | tester:test:80 -      loss:  0.1853
2023-08-14 12:14:57.719 | INFO     | tester:test:84 - AUC  : 0.9631
2023-08-14 12:14:57.720 | INFO     | tester:test:84 - F1   : 0.6995
2023-08-14 12:14:57.720 | INFO     | tester:test:84 - Acc  : 0.9875
2023-08-14 12:14:57.721 | INFO     | tester:test:84 - Sen  : 0.6492
2023-08-14 12:14:57.721 | INFO     | tester:test:84 - Spe  : 0.9958
2023-08-14 12:14:57.722 | INFO     | tester:test:84 - pre  : 0.8023
2023-08-14 12:14:57.722 | INFO     | tester:test:84 - IOU  : 0.5645
2023-08-14 12:14:57.722 | INFO     | tester:test:84 - MCC  : 0.7054
TEST (10) | Loss: 0.1242 | AUC 0.9847 F1 0.82

In [18]:
main_result(config_file, weight_path=check_path, quality="IC", value=0)

main_result(config_file, weight_path=check_path, quality="IC", value=1)


TEST (51) | Loss: 0.1224 | AUC 0.9816 F1 0.8244 Acc 0.9858  Sen 0.7873 Spe 0.9949 Pre 0.8810 IOU 0.7187 MCC 0.8217 |B 0.03 D 0.02 |: 100%|██████████| 52/52 [00:09<00:00,  5.69it/s]
2023-08-14 12:15:45.544 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:15:45.545 | INFO     | tester:test:79 - test time:  0.0344
2023-08-14 12:15:45.545 | INFO     | tester:test:80 -      loss:  0.1224
2023-08-14 12:15:45.546 | INFO     | tester:test:84 - AUC  : 0.9816
2023-08-14 12:15:45.546 | INFO     | tester:test:84 - F1   : 0.8244
2023-08-14 12:15:45.547 | INFO     | tester:test:84 - Acc  : 0.9858
2023-08-14 12:15:45.548 | INFO     | tester:test:84 - Sen  : 0.7873
2023-08-14 12:15:45.548 | INFO     | tester:test:84 - Spe  : 0.9949
2023-08-14 12:15:45.548 | INFO     | tester:test:84 - pre  : 0.881
2023-08-14 12:15:45.549 | INFO     | tester:test:84 - IOU  : 0.7187
2023-08-14 12:15:45.550 | INFO     | tester:test:84 - MCC  : 0.8217
TEST (147) | Loss: 0.0771 | AUC 0.9923 F1 0.90

In [19]:
main_result(config_file, weight_path=check_path, quality="Blur", value=0)

main_result(config_file, weight_path=check_path, quality="Blur", value=1)


TEST (40) | Loss: 0.1373 | AUC 0.9788 F1 0.7925 Acc 0.9870  Sen 0.7458 Spe 0.9959 Pre 0.8646 IOU 0.6739 MCC 0.7920 |B 0.05 D 0.04 |: 100%|██████████| 41/41 [00:07<00:00,  5.40it/s]
2023-08-14 12:16:20.260 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:16:20.261 | INFO     | tester:test:79 - test time:  0.0512
2023-08-14 12:16:20.262 | INFO     | tester:test:80 -      loss:  0.1373
2023-08-14 12:16:20.262 | INFO     | tester:test:84 - AUC  : 0.9788
2023-08-14 12:16:20.263 | INFO     | tester:test:84 - F1   : 0.7925
2023-08-14 12:16:20.263 | INFO     | tester:test:84 - Acc  : 0.987
2023-08-14 12:16:20.264 | INFO     | tester:test:84 - Sen  : 0.7458
2023-08-14 12:16:20.264 | INFO     | tester:test:84 - Spe  : 0.9959
2023-08-14 12:16:20.265 | INFO     | tester:test:84 - pre  : 0.8646
2023-08-14 12:16:20.265 | INFO     | tester:test:84 - IOU  : 0.6739
2023-08-14 12:16:20.266 | INFO     | tester:test:84 - MCC  : 0.792
TEST (158) | Loss: 0.0764 | AUC 0.9923 F1 0.909

In [20]:
main_result(config_file, weight_path=check_path, quality="LC", value=0)

main_result(config_file, weight_path=check_path, quality="LC", value=1)


TEST (14) | Loss: 0.1853 | AUC 0.9631 F1 0.6995 Acc 0.9875  Sen 0.6492 Spe 0.9958 Pre 0.8023 IOU 0.5645 MCC 0.7054 |B 0.09 D 0.08 |: 100%|██████████| 15/15 [00:03<00:00,  4.24it/s]
2023-08-14 12:16:52.907 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:16:52.908 | INFO     | tester:test:79 - test time:  0.0922
2023-08-14 12:16:52.908 | INFO     | tester:test:80 -      loss:  0.1853
2023-08-14 12:16:52.909 | INFO     | tester:test:84 - AUC  : 0.9631
2023-08-14 12:16:52.909 | INFO     | tester:test:84 - F1   : 0.6995
2023-08-14 12:16:52.910 | INFO     | tester:test:84 - Acc  : 0.9875
2023-08-14 12:16:52.910 | INFO     | tester:test:84 - Sen  : 0.6492
2023-08-14 12:16:52.911 | INFO     | tester:test:84 - Spe  : 0.9958
2023-08-14 12:16:52.911 | INFO     | tester:test:84 - pre  : 0.8023
2023-08-14 12:16:52.912 | INFO     | tester:test:84 - IOU  : 0.5645
2023-08-14 12:16:52.912 | INFO     | tester:test:84 - MCC  : 0.7054
TEST (184) | Loss: 0.0810 | AUC 0.9916 F1 0.9

## WNet

In [21]:
config_file = '../configs/lwnet_dicebce.yaml'
check_path = "/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/WNet/FIVES/CE_DiceLoss/checkpoint-epoch85.pth"


In [22]:
main_result(config_file, weight_path=check_path, quality="Score", value=0)

main_result(config_file, weight_path=check_path, quality="Score", value=1)

main_result(config_file, weight_path=check_path, quality="Score", value=2)

main_result(config_file, weight_path=check_path, quality="Score", value=3)

TEST (14) | Loss: 0.6397 | AUC 0.9035 F1 0.5046 Acc 0.9822  Sen 0.3801 Spe 0.9986 Pre 0.8393 IOU 0.3610 MCC 0.5429 |B 0.09 D 0.08 |: 100%|██████████| 15/15 [00:03<00:00,  4.30it/s]
2023-08-14 12:17:27.240 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:17:27.241 | INFO     | tester:test:79 - test time:  0.0915
2023-08-14 12:17:27.242 | INFO     | tester:test:80 -      loss:  0.6397
2023-08-14 12:17:27.242 | INFO     | tester:test:84 - AUC  : 0.9035
2023-08-14 12:17:27.243 | INFO     | tester:test:84 - F1   : 0.5046
2023-08-14 12:17:27.243 | INFO     | tester:test:84 - Acc  : 0.9822
2023-08-14 12:17:27.244 | INFO     | tester:test:84 - Sen  : 0.3801
2023-08-14 12:17:27.245 | INFO     | tester:test:84 - Spe  : 0.9986
2023-08-14 12:17:27.245 | INFO     | tester:test:84 - pre  : 0.8393
2023-08-14 12:17:27.246 | INFO     | tester:test:84 - IOU  : 0.361
2023-08-14 12:17:27.246 | INFO     | tester:test:84 - MCC  : 0.5429
TEST (10) | Loss: 0.4379 | AUC 0.9674 F1 0.711

In [23]:
main_result(config_file, weight_path=check_path, quality="IC", value=0)

main_result(config_file, weight_path=check_path, quality="IC", value=1)


TEST (51) | Loss: 0.4263 | AUC 0.9568 F1 0.7177 Acc 0.9794  Sen 0.6422 Spe 0.9943 Pre 0.8747 IOU 0.5882 MCC 0.7260 |B 0.03 D 0.02 |: 100%|██████████| 52/52 [00:08<00:00,  5.79it/s]
2023-08-14 12:18:08.226 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:18:08.227 | INFO     | tester:test:79 - test time:  0.0339
2023-08-14 12:18:08.227 | INFO     | tester:test:80 -      loss:  0.4263
2023-08-14 12:18:08.228 | INFO     | tester:test:84 - AUC  : 0.9568
2023-08-14 12:18:08.228 | INFO     | tester:test:84 - F1   : 0.7177
2023-08-14 12:18:08.229 | INFO     | tester:test:84 - Acc  : 0.9794
2023-08-14 12:18:08.230 | INFO     | tester:test:84 - Sen  : 0.6422
2023-08-14 12:18:08.230 | INFO     | tester:test:84 - Spe  : 0.9943
2023-08-14 12:18:08.231 | INFO     | tester:test:84 - pre  : 0.8747
2023-08-14 12:18:08.232 | INFO     | tester:test:84 - IOU  : 0.5882
2023-08-14 12:18:08.232 | INFO     | tester:test:84 - MCC  : 0.726
TEST (147) | Loss: 0.2771 | AUC 0.9867 F1 0.85

In [24]:
main_result(config_file, weight_path=check_path, quality="Blur", value=0)

main_result(config_file, weight_path=check_path, quality="Blur", value=1)


TEST (40) | Loss: 0.4992 | AUC 0.9470 F1 0.6499 Acc 0.9805  Sen 0.5422 Spe 0.9969 Pre 0.8730 IOU 0.5043 MCC 0.6673 |B 0.04 D 0.03 |: 100%|██████████| 41/41 [00:07<00:00,  5.47it/s]
2023-08-14 12:18:39.301 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:18:39.301 | INFO     | tester:test:79 - test time:  0.0392
2023-08-14 12:18:39.302 | INFO     | tester:test:80 -      loss:  0.4992
2023-08-14 12:18:39.303 | INFO     | tester:test:84 - AUC  : 0.947
2023-08-14 12:18:39.304 | INFO     | tester:test:84 - F1   : 0.6499
2023-08-14 12:18:39.305 | INFO     | tester:test:84 - Acc  : 0.9805
2023-08-14 12:18:39.305 | INFO     | tester:test:84 - Sen  : 0.5422
2023-08-14 12:18:39.306 | INFO     | tester:test:84 - Spe  : 0.9969
2023-08-14 12:18:39.306 | INFO     | tester:test:84 - pre  : 0.873
2023-08-14 12:18:39.307 | INFO     | tester:test:84 - IOU  : 0.5043
2023-08-14 12:18:39.308 | INFO     | tester:test:84 - MCC  : 0.6673
TEST (158) | Loss: 0.2686 | AUC 0.9872 F1 0.865

In [25]:
main_result(config_file, weight_path=check_path, quality="LC", value=0)

main_result(config_file, weight_path=check_path, quality="LC", value=1)


TEST (14) | Loss: 0.6397 | AUC 0.9035 F1 0.5046 Acc 0.9822  Sen 0.3801 Spe 0.9986 Pre 0.8393 IOU 0.3610 MCC 0.5429 |B 0.08 D 0.07 |: 100%|██████████| 15/15 [00:03<00:00,  4.30it/s]
2023-08-14 12:19:07.873 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:19:07.874 | INFO     | tester:test:79 - test time:  0.083
2023-08-14 12:19:07.874 | INFO     | tester:test:80 -      loss:  0.6397
2023-08-14 12:19:07.875 | INFO     | tester:test:84 - AUC  : 0.9035
2023-08-14 12:19:07.876 | INFO     | tester:test:84 - F1   : 0.5046
2023-08-14 12:19:07.876 | INFO     | tester:test:84 - Acc  : 0.9822
2023-08-14 12:19:07.877 | INFO     | tester:test:84 - Sen  : 0.3801
2023-08-14 12:19:07.878 | INFO     | tester:test:84 - Spe  : 0.9986
2023-08-14 12:19:07.878 | INFO     | tester:test:84 - pre  : 0.8393
2023-08-14 12:19:07.879 | INFO     | tester:test:84 - IOU  : 0.361
2023-08-14 12:19:07.879 | INFO     | tester:test:84 - MCC  : 0.5429
TEST (184) | Loss: 0.2896 | AUC 0.9851 F1 0.846

## UNet

In [26]:
config_file = '../configs/unet_dicebce.yaml'
check_path = "/mnt/qb/berens/users/jfadugba97/RetinaSegmentation/model_results/Unet/FIVES/CE_DiceLoss/checkpoint-epoch70.pth"


In [27]:
main_result(config_file, weight_path=check_path, quality="Score", value=0)

main_result(config_file, weight_path=check_path, quality="Score", value=1)

main_result(config_file, weight_path=check_path, quality="Score", value=2)

main_result(config_file, weight_path=check_path, quality="Score", value=3)

TEST (14) | Loss: 0.1949 | AUC 0.9620 F1 0.6831 Acc 0.9868  Sen 0.6327 Spe 0.9960 Pre 0.8051 IOU 0.5498 MCC 0.6920 |B 0.10 D 0.08 |: 100%|██████████| 15/15 [00:03<00:00,  4.17it/s]
2023-08-14 12:19:44.575 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:19:44.576 | INFO     | tester:test:79 - test time:  0.0955
2023-08-14 12:19:44.576 | INFO     | tester:test:80 -      loss:  0.1949
2023-08-14 12:19:44.577 | INFO     | tester:test:84 - AUC  : 0.962
2023-08-14 12:19:44.577 | INFO     | tester:test:84 - F1   : 0.6831
2023-08-14 12:19:44.578 | INFO     | tester:test:84 - Acc  : 0.9868
2023-08-14 12:19:44.578 | INFO     | tester:test:84 - Sen  : 0.6327
2023-08-14 12:19:44.579 | INFO     | tester:test:84 - Spe  : 0.996
2023-08-14 12:19:44.580 | INFO     | tester:test:84 - pre  : 0.8051
2023-08-14 12:19:44.580 | INFO     | tester:test:84 - IOU  : 0.5498
2023-08-14 12:19:44.581 | INFO     | tester:test:84 - MCC  : 0.692
TEST (10) | Loss: 0.1224 | AUC 0.9856 F1 0.8280 

In [28]:
main_result(config_file, weight_path=check_path, quality="IC", value=0)

main_result(config_file, weight_path=check_path, quality="IC", value=1)


TEST (51) | Loss: 0.1234 | AUC 0.9817 F1 0.8217 Acc 0.9859  Sen 0.7869 Spe 0.9949 Pre 0.8805 IOU 0.7175 MCC 0.8197 |B 0.04 D 0.03 |: 100%|██████████| 52/52 [00:09<00:00,  5.74it/s]
2023-08-14 12:20:31.045 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:20:31.045 | INFO     | tester:test:79 - test time:  0.0387
2023-08-14 12:20:31.046 | INFO     | tester:test:80 -      loss:  0.1234
2023-08-14 12:20:31.046 | INFO     | tester:test:84 - AUC  : 0.9817
2023-08-14 12:20:31.047 | INFO     | tester:test:84 - F1   : 0.8217
2023-08-14 12:20:31.047 | INFO     | tester:test:84 - Acc  : 0.9859
2023-08-14 12:20:31.048 | INFO     | tester:test:84 - Sen  : 0.7869
2023-08-14 12:20:31.048 | INFO     | tester:test:84 - Spe  : 0.9949
2023-08-14 12:20:31.049 | INFO     | tester:test:84 - pre  : 0.8805
2023-08-14 12:20:31.049 | INFO     | tester:test:84 - IOU  : 0.7175
2023-08-14 12:20:31.049 | INFO     | tester:test:84 - MCC  : 0.8197
TEST (147) | Loss: 0.0747 | AUC 0.9926 F1 0.9

In [29]:
main_result(config_file, weight_path=check_path, quality="Blur", value=0)

main_result(config_file, weight_path=check_path, quality="Blur", value=1)


TEST (40) | Loss: 0.1396 | AUC 0.9788 F1 0.7882 Acc 0.9868  Sen 0.7487 Spe 0.9955 Pre 0.8580 IOU 0.6709 MCC 0.7882 |B 0.04 D 0.03 |: 100%|██████████| 41/41 [00:07<00:00,  5.72it/s]
2023-08-14 12:21:04.579 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:21:04.579 | INFO     | tester:test:79 - test time:  0.0401
2023-08-14 12:21:04.580 | INFO     | tester:test:80 -      loss:  0.1396
2023-08-14 12:21:04.580 | INFO     | tester:test:84 - AUC  : 0.9788
2023-08-14 12:21:04.580 | INFO     | tester:test:84 - F1   : 0.7882
2023-08-14 12:21:04.582 | INFO     | tester:test:84 - Acc  : 0.9868
2023-08-14 12:21:04.583 | INFO     | tester:test:84 - Sen  : 0.7487
2023-08-14 12:21:04.583 | INFO     | tester:test:84 - Spe  : 0.9955
2023-08-14 12:21:04.584 | INFO     | tester:test:84 - pre  : 0.858
2023-08-14 12:21:04.584 | INFO     | tester:test:84 - IOU  : 0.6709
2023-08-14 12:21:04.584 | INFO     | tester:test:84 - MCC  : 0.7882
TEST (158) | Loss: 0.0739 | AUC 0.9926 F1 0.91

In [30]:
main_result(config_file, weight_path=check_path, quality="LC", value=0)

main_result(config_file, weight_path=check_path, quality="LC", value=1)


TEST (14) | Loss: 0.1949 | AUC 0.9620 F1 0.6831 Acc 0.9868  Sen 0.6327 Spe 0.9960 Pre 0.8051 IOU 0.5498 MCC 0.6920 |B 0.11 D 0.10 |: 100%|██████████| 15/15 [00:03<00:00,  4.12it/s]
2023-08-14 12:21:36.146 | INFO     | tester:test:78 - ###### TEST EVALUATION ######
2023-08-14 12:21:36.147 | INFO     | tester:test:79 - test time:  0.1082
2023-08-14 12:21:36.147 | INFO     | tester:test:80 -      loss:  0.1949
2023-08-14 12:21:36.148 | INFO     | tester:test:84 - AUC  : 0.962
2023-08-14 12:21:36.148 | INFO     | tester:test:84 - F1   : 0.6831
2023-08-14 12:21:36.149 | INFO     | tester:test:84 - Acc  : 0.9868
2023-08-14 12:21:36.149 | INFO     | tester:test:84 - Sen  : 0.6327
2023-08-14 12:21:36.151 | INFO     | tester:test:84 - Spe  : 0.996
2023-08-14 12:21:36.151 | INFO     | tester:test:84 - pre  : 0.8051
2023-08-14 12:21:36.152 | INFO     | tester:test:84 - IOU  : 0.5498
2023-08-14 12:21:36.152 | INFO     | tester:test:84 - MCC  : 0.692
TEST (184) | Loss: 0.0787 | AUC 0.9920 F1 0.9033